In [20]:
import numpy as np
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader,Dataset
from math import sqrt
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split
import copy
from collections import defaultdict
import matplotlib.pyplot as plt
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [21]:
def extract_data(path,create_test=None):
    X_train=[]
    y_train=[]
    X_test=[]
    y_test=[]

    file=os.listdir(path)

    for entity in file:
        try:
            entityPath=os.listdir(f"{path}/{entity}")
            for i,data in enumerate(entityPath):
                try:
                    current_img=cv2.imread(f"{path}/{entity}/{data}")
                    if current_img is not None:
                        if (create_test is not None and ((len(entityPath)>1) and (i==0))):
                            X_test.append(current_img)
                            y_test.append(entity)
                        else:
                            X_train.append(current_img)
                            y_train.append(entity)
                    else:
                        print(f"{path}/{entity}/{data} is None type")
                except Exception as e:
                    print(f"Erreur avec {path}/{entity}/{data} : {e}")
        except Exception as ex:
            print(f"Erreur avec {path}/{entity} : {ex}")
    
    print(f"Number of training sample : {len(X_train)}\n")
    if create_test is not None:
        print(f"Number of test sample : {len(X_test)}\n")
        return X_train,y_train,X_test,y_test
    else:
        return X_train,y_train

In [22]:
transform_face_test=transforms.Compose([
    transforms.ToPILImage(), #transforme en format PIL
    transforms.Resize((100,100)),
    transforms.ToTensor() #reconvertit l'img en format tensor
])

transform_face=transforms.Compose([
    transforms.ToPILImage(), #transforme en format PIL
    transforms.Resize((100,100)),
    transforms.RandomHorizontalFlip(), #0.5 de proba d'inverser la gauche et la droite de l'img pour rendre le modèle invariant à la symétrie
    transforms.RandomRotation(15), #applique rota random entre -10° et +10° 
    transforms.ColorJitter(brightness=0.3,contrast=0.3,saturation=0.2,hue=0.02), #altère aléatoirement la luminosité et le contraste de l'image
    transforms.ToTensor() #reconvertit l'img en format tensor
])

transform_eyes=transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((50,50)),
    transforms.ToTensor()
])

transform_eyes_test=transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((50,50)),
    transforms.ToTensor()
])

In [ ]:
def extract_face_and_eyes(img):
    if not isinstance(img,np.ndarray):
        img=cv2.imread(img)

    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(60,60)
    )

    if len(faces)==0:
        return [],[]
    
    x,y,w,h=faces[0]
    face=img[y:y+h,x:x+w]
    eyes=eye_cascade.detectMultiScale(cv2.cvtColor(face,cv2.COLOR_BGR2GRAY))
    if len(eyes)==2:
        if eyes[0][0]<eyes[1][0]:
            x2=eyes[0][0]
            w2=eyes[1][0]+eyes[1][2]-x2
        else:
            x2=eyes[1][0]
            w2=eyes[0][0]+eyes[0][2]-x2
        if eyes[0][1]<eyes[1][1]:
            y2=eyes[0][1]
            h2=eyes[1][1]+eyes[1][3]-y2
        else:
            y2=eyes[1][1]
            h2=eyes[0][1]+eyes[0][3]-y2
        imgEyes=face[y2:y2+h2,x2:x2+w2]
        return face,imgEyes
    else:
        return face,[]
    

In [24]:
def createData(X):
    X_face=[]
    X_eyes=[]
    for i,x in enumerate(X):
        face,eyes=extract_face_and_eyes(x)
        X_face.append(face)
        if len(eyes)!=0:
            X_eyes.append(eyes)
        else:
            X_eyes.append(None)
    return X_face,X_eyes

In [25]:
class FaceEyesDataset(Dataset):
    def __init__(self,faces,eyes,labels):
        self.faces=faces
        self.eyes=eyes
        self.labels=labels

    def __len__(self):
        return len(self.faces)
    
    def __getitem__(self,idx):
        return self.faces[idx],self.eyes[idx],self.labels[idx]

def fusion_train(X_train,y_train,nb_epoch,labels,batch_size,models=None,patience=5,val_split=False):
    nb_classes=len(list(set(y_train)))
    if not models:
        facemodel,eyesmodel,fusionModel=feature_fusionModel(nb_classes)
    else:
        fusionModel,facemodel,eyesmodel=models
    
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    fusionModel.to(device)
    facemodel.to(device)
    eyesmodel.to(device)

    X_face,X_eyes=createData(X_train)
    faceslist=([transform_face(x) for x in X_face])
    ytorchlist=([labels[y] for y in y_train])
    eyeslist=[]
    for eye in X_eyes:
        if eye:
            eyeslist.append(transform_eyes(eye))
        else:
            eyeslist.append(torch.zeros((1,50,50)))

    if val_split:
        faceslist,faces_val,eyeslist,eyes_val,ytorchlist,y_val=train_test_split(faceslist,eyeslist,ytorchlist,test_size=0.2,random_state=1,stratify=ytorchlist)
        faces_val=torch.stack(faces_val)
        eyes_val=torch.stack(eyes_val)
        y_val=torch.tensor(y_val,dtype=torch.long)
        val_dataset=FaceEyesDataset(faces_val,eyes_val,y_val)
        val_loader=DataLoader(val_dataset,batch_size=batch_size,shuffle=False)
        best_val_acc=0
        wait=0

    faces=torch.stack(faceslist)
    eyes=torch.stack(eyeslist)
    ytorch=torch.tensor(ytorchlist,dtype=torch.long)

    train_dataset=FaceEyesDataset(faces,eyes,ytorch)
    train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)

    criterion=torch.nn.CrossEntropyLoss()
    optimizer=torch.optim.Adam(list(facemodel.parameters())+list(eyesmodel.parameters())+list(fusionModel.parameters()), lr=0.001)
    #on veut optimiser les poids des trois modèles en même temps
    tabTrain=[]
    tabVal=[]
    for epoch in range(nb_epoch):
        fusionModel.train()
        facemodel.train()
        eyesmodel.train()

        total_loss=0.0
        correct=0
        total=0

        for face_batch,eyes_batch,y_batch in train_loader:
            face_batch=face_batch.to(device)
            eye_batch=eye_batch.to(device)
            y_batch=y_batch.to(device)

            optimizer.zero_grad()

            featuresFaces=facemodel(face_batch)
            featuresEyes=eyesmodel(eyes_batch)
            featuresFusion=torch.cat((featuresFaces,featuresEyes),dim=1)
            outputs=fusionModel(featuresFusion)

            loss=criterion(outputs,y_batch)
            loss.backward()
            optimizer.step()

            total_loss+=loss.item()
            _,predicted=torch.max(outputs,1)
            correct+=(predicted==y_batch).sum().item()
            total+=y_batch.size(0)
        
        acc=100*correct/total
        tabTrain.append(acc)
        print(f"\nEpoch : {epoch+1}/{nb_epoch}\n Perte :{total_loss:.4f}\n Précision : {acc:.2f}%")

        if val_split:
            true,false,tot=fusionEvaluate((facemodel,eyesmodel,fusionModel),val_loader,labels)
            val_acc=sum(true.values())/sum(tot.values())
            tabVal.append(100*val_acc)
            if val_acc>best_val_acc:
                best_val_acc=val_acc #si l'acc de la val est meilleure on l'assigne et on reset le timer 
                wait=0
                best_model_faces=copy.deepcopy(facemodel.state_dict())
                best_model_eyes=copy.deepcopy(eyesmodel.state_dict())
                best_model_fusion=copy.deepcopy(fusionModel.state_dict())
            else:
                if wait>=patience: #on attend le nombre d'epoch max sans amélioration
                    print("Early stopping activated")
                    break
                wait+=1
            print(f"\n Val accuracy : {val_acc*100:.2f}%\n Série sans amélioration : {wait}")
    facemodel.load_state_dict(best_model_faces)
    eyesmodel.load_state_dict(best_model_eyes)
    fusionModel.load_state_dict(best_model_fusion)
        
    absc=list(range(len(tabTrain)))
    plt.figure(figsize=(8,5))
    plt.plot(absc,tabTrain,label="Training",marker="o")
    if val_split is not None:
        plt.plot(absc,tabVal,label="Validation",marker="s")
    plt.xlim(1,len(tabTrain))
    plt.ylim(0,100)
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy (%)")
    plt.legend()
    plt.grid(True)
    print("Training terminé.")
    plt.show()
    return facemodel,eyesmodel,fusionModel

In [26]:
def fusionEvaluate(models,loader,labels,graph=False):
    if device is None:
        device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

    facemodel,eyesmodel,fusionmodel=models
    facemodel.to(device)
    facemodel.eval()
    eyesmodel.to(device)
    eyesmodel.eval()
    fusionmodel.to(device)
    fusionmodel.eval()

    labelF=defaultdict(int)
    labelC=defaultdict(int)
    labelT=defaultdict(int)

    with torch.no_grad():
        for faces_batch,eyes_batch,y_batch in loader:
            faces_batch=faces_batch.to(device)
            eyes_batch=eyes_batch.to(device)
            y_batch=y_batch.to(device)

            featuresFaces=facemodel(faces_batch)
            featuresEyes=eyesmodel(eyes_batch)
            featuresFusion=torch.cat((featuresFaces,featuresEyes),dim=1)
            outputs=fusionmodel(featuresFusion)

            _,pred=torch.max(outputs,1)
            for i in range(len(y_batch)):
                label=y_batch[i].item()
                pred[i]=pred[i].item()

                if pred[i]==label:
                    labelC[label]+=1
                else:
                    labelF[label]+=1
                labelT[label]+=1

    if graph:
        print(f"Total precision : {sum(labelC.values())/sum(labelT.values())*100:.2f}%")
        precisions={}
        for label in labelT:
            tot=labelT[label]
            correct=labelC.get(label,0)
            precisions[label]=correct/tot if tot>0 else 0
        prec_sorted=sorted(precisions.items(),key=lambda x: x[1])
        labels_ids,values= zip(*prec_sorted)

        id2label = {v: k for k, v in labels.items()}
        class_names=[id2label[i] for i in labels_ids]

        plt.figure(figsize=(12, 5))
        plt.bar(class_names, values, color='skyblue')
        plt.xticks(rotation=45)
        plt.ylabel("Précision par classe")
        plt.title("Performance du modèle par classe")
        plt.tight_layout()
        plt.show()
    
    return labelC,labelF,labelT

In [27]:
def feature_fusionModel(nb_classes):
    face_model=nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=3, padding=1),   # (1,100,100) -> (16,100,100)
        nn.ReLU(),
        nn.MaxPool2d(2, 2),                           # (16,50,50)

        nn.Conv2d(16, 32, kernel_size=3, padding=1),  # (32,50,50)
        nn.ReLU(),
        nn.MaxPool2d(2, 2),                           # (32,25,25)

        nn.Conv2d(32, 64, kernel_size=3, padding=1),  # (64,25,25)
        nn.ReLU(),
        nn.MaxPool2d(2, 2),                           # (64,12,12)

        nn.Flatten(),                                 # 64*12*12 = 9216
        nn.Linear(9216, 512),
        nn.ReLU()
    )

    eyes_model=nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=3, padding=1),   # (1,50,50) -> (16,50,50)
        nn.ReLU(),
        nn.MaxPool2d(2, 2),                           # (16,25,25)

        nn.Conv2d(16, 32, kernel_size=3, padding=1),  # (32,25,25)
        nn.ReLU(),
        nn.MaxPool2d(2, 2),                           # (32,12,12)

        nn.Flatten(),                                 # 32*12*12 = 4608
        nn.Linear(4608, 256),
        nn.ReLU()
    )

    final_classifier=nn.Sequential(
        nn.Linear(768, 256),
        nn.ReLU(),
        nn.Dropout(0.5),  # Dropout pour éviter l'overfitting

        nn.Linear(256, nb_classes)
    )

    return face_model,eyes_model,final_classifier

In [17]:
X_train,y_train=extract_data("DataFaces")
labels={label:i for i,label in enumerate(sorted(set(y_train)))}

Number of training sample : 17534



In [18]:
X_train,X_test,y_train,y_test=train_test_split(X_train,y_train,test_size=0.2,random_state=1,stratify=y_train)

In [28]:
facemodel,eyesmodel,fusionmodel=fusion_train(X_train,y_train,100,labels,32,patience=5,val_split=True)

TypeError: pic should be Tensor or ndarray. Got <class 'list'>.

In [ ]:
facestest,eyestest=extract_face_and_eyes(X_test)
labelC,labelF,labelT=fusionEvaluate((facemodel,eyesmodel,fusionmodel),)

In [ ]:
def fusionPipeline(X,y):
    